In [120]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-recommender-system/BX-Books.csv
/kaggle/input/book-recommender-system/BX-Book-Ratings.csv
/kaggle/input/book-recommender-system/BX-Users.csv


In [121]:
books = pd.read_csv("/kaggle/input/book-recommender-system/BX-Books.csv", sep=';', encoding="latin-1", error_bad_lines=False)
users = pd.read_csv("/kaggle/input/book-recommender-system/BX-Users.csv", sep=';', encoding="latin-1", error_bad_lines=False)
ratings = pd.read_csv("/kaggle/input/book-recommender-system/BX-Book-Ratings.csv", sep=';', encoding="latin-1", error_bad_lines=False)

print(books.head(5))
print(users.head(5))
print(ratings.head(5))

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 

         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/

In [122]:
print("---Shape of the datasets---")
print("Books:", books.shape)
print("Users:", users.shape)
print("Ratings:", ratings.shape)
print("--------------------------------------")
print("---Columns of the datasets---")
print("Books:", list(books.columns))
print("Users:", list(users.columns))
print("Ratings:", list(ratings.columns))

---Shape of the datasets---
Books: (271360, 8)
Users: (278858, 3)
Ratings: (1149780, 3)
--------------------------------------
---Columns of the datasets---
Books: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
Users: ['User-ID', 'Location', 'Age']
Ratings: ['User-ID', 'ISBN', 'Book-Rating']


In [123]:
# Clean data - removing unwanted columns
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Publisher']]
users = users[['User-ID', 'Age']]
ratings = ratings[['User-ID', 'ISBN', 'Book-Rating']]

In [124]:
books['Book-Title'] = books['Book-Title'].str.lower()
books['Book-Author'] = books['Book-Author'].str.lower()
books['Publisher'] = books['Publisher'].str.lower()

print(books.head(2))

         ISBN           Book-Title           Book-Author  \
0  0195153448  classical mythology    mark p. o. morford   
1  0002005018         clara callan  richard bruce wright   

                 Publisher  
0  oxford university press  
1    harperflamingo canada  


In [125]:
#fixing error(can only concatenate str (not "float") to str) with following code
# Replacing NaN values with an empty string
books['Book-Title'] = books['Book-Title'].fillna(' ')
books['Book-Author'] = books['Book-Author'].fillna(' ')
books['Publisher'] = books['Publisher'].fillna(' ')

users['Age'] = users['Age'].fillna(' ')

# Combine the book titles, authors, and publishers into a single string
# lambda: we do not use this function anywhere else.
books['book_oneStr'] = books.apply(lambda row: row['Book-Title'] + ' ' + row['Book-Author'] + ' ' + row['Publisher'], axis=1)

In [126]:
# renaming some of the tables columns for easy access
books = books.rename(columns = {'Book-Title':'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher':'publisher'})
users = users.rename(columns = {'User-ID':'uid', 'Location':'location', 'Age':'age'})
ratings = ratings.rename(columns = {'User-ID':'uid', 'Book-Rating':'rating'})

books.head()

,ISBN,title,author,publisher,book_oneStr
0,0195153448,classical mythology,mark p. o. morford,oxford university press,classical mythology mark p. o. morford oxford ...
1,0002005018,clara callan,richard bruce wright,harperflamingo canada,clara callan richard bruce wright harperflamin...
2,0060973129,decision in normandy,carlo d'este,harperperennial,decision in normandy carlo d'este harperperennial
3,0374157065,flu: the story of the great influenza pandemic...,gina bari kolata,farrar straus giroux,flu: the story of the great influenza pandemic...
4,0393045218,the mummies of urumchi,e. j. w. barber,w. w. norton &amp; company,the mummies of urumchi e. j. w. barber w. w. n...


In [127]:
print("There is Nan value:", books.isna().any())
print("There is Nan value:", users.isna().any())
print("There is Nan value:", ratings.isna().any())

# Merge data
book_ratings = pd.merge(ratings, books, on='ISBN')
user_ratings = pd.merge(ratings, users, on='uid')

print(book_ratings)
print("-------------------------------------------------------------------------------------")
print(user_ratings)

There is Nan value: ISBN           False
title          False
author         False
publisher      False
book_oneStr    False
dtype: bool
There is Nan value: uid    False
age    False
dtype: bool
There is Nan value: uid       False
ISBN      False
rating    False
dtype: bool
            uid        ISBN  rating  \
0        276725  034545104X       0   
1          2313  034545104X       5   
2          6543  034545104X       0   
3          8680  034545104X       5   
4         10314  034545104X       9   
...         ...         ...     ...   
1031131  276688  0517145553       0   
1031132  276688  1575660792       7   
1031133  276690  0590907301       0   
1031134  276704  0679752714       0   
1031135  276704  0806917695       5   

                                                     title           author  \
0                                     flesh tones: a novel       m. j. rose   
1                                     flesh tones: a novel       m. j. rose   
2                  

In [128]:
# Define book embeddings using Doc2Vec

#creating a list of tagged documents, where each document corresponds to a book title
#TaggedDocument is a class from gensim package which takes a list of words and unique int ID
#the list is the book title split by whitespace
#the ID is the index of the book in the books dataframe
#documents = [TaggedDocument(book['title'].split(), [i]) for i, book in books.iterrows()]

#creating a Doc2Vec model using the tagged documents created above
#vector_size is the dimensionality of the resulting document vectors (in this case, 50). 
#window is the maximum distance between the current and predicted word within a sentence. 
#min_count is the minimum count of words to consider when training the model. 
#workers is the number of threads to use when training the model. 
#epochs is the number of iterations over the corpus during training
#model = Doc2Vec(documents, vector_size=50, window=2, min_count=1, workers=4, epochs=20, callbacks=None)

#creating an array of book embeddings using the trained Doc2Vec model. 
#Each book is represented as a 50-dimensional vector. 
#The array is created by iterating over the indices of the books in the books dataframe 
#and calling the docvecs property of the model to get the corresponding document vector. 
#The resulting list of vectors is converted to a numpy array
#book_embeddings = np.array([model.docvecs[i] for i in range(len(books))])


# Define book embeddings using Doc2Vec
documents = []
for i, book in books.iterrows():
    t = book['title']
    document = TaggedDocument(t.split(), [i])
    documents.append(document)
    #print(document)
model = Doc2Vec(vector_size=50, window=2, min_count=1, workers=4)
model.build_vocab(documents, progress_per=10000)
model.train(documents, total_examples=model.corpus_count, epochs=100)

book_embeddings = np.array([model.docvecs[i] for i in range(len(books))]) 



# Define user embeddings based on average book ratings
user_embeddings = []
for i, user in users.iterrows():
    user_ratings = user_ratings[user_ratings['uid'] == user['uid']]
    if len(user_ratings) > 0:
        user_embedding = np.mean(book_embeddings[np.array([books['title'] == title for title in user_ratings['title']])], axis=0)
    else:
        user_embedding = np.zeros((50,))
    user_embeddings.append(user_embedding)
user_embeddings = np.array(user_embeddings)



# Define function to get book recommendations based on input
def get_recommendations(user_input):
    input_embedding = model.infer_vector(user_input.split())
    similarities = cosine_similarity([input_embedding], book_embeddings)[0]
    sorted_indices = np.argsort(similarities)[::-1]
    recommended_books = books.iloc[sorted_indices[:5]]
    return recommended_books


# Testing 1
user_input = 'Harry Potter'
recommendations = get_recommendations(user_input)
print("Here are the Top 5 books recommendation for you:")
print(recommendations)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).


Here are the Top 5 books recommendation for you:
              ISBN                                              title  \
158208  0789301121                             sleep : bedtime reader   
50279   0671036181                               the last lovely city   
43019   0375700706  an american dream (vintage international (pape...   
252503  037305615X  valentine street hustle (harlequin desire, no ...   
162882  0671675869   the gift horse's mouth: a jimmy flannery mystery   

                        author                publisher  \
158208          robert peacock      universe publishing   
50279              alice adams  washington square press   
43019            norman mailer        vintage books usa   
252503            ryanne corey               silhouette   
162882  robert wright campbell             pocket books   

                                              book_oneStr  
158208  sleep : bedtime reader robert peacock universe...  
50279   the last lovely city alice ad

In [129]:
# Testing 2
user_input = 'Thriller'
recommendations = get_recommendations(user_input)
print("Here are the Top 5 books recommendation for you:")
print(recommendations)

Here are the Top 5 books recommendation for you:
              ISBN                                              title  \
64368   0373106076               enter my jungle (harlequin presents)   
88742   1555833691  doc and fluff: the dystopian tale of a girl an...   
221933  1562880179                         jungle jokes &amp; riddles   
192145  0816735425            the temptation (midnight secrets, no 1)   
129214  0380815486                               my lady's temptation   

                   author             publisher  \
64368   lindsay armstrong             harlequin   
88742         pat califia   alyson publications   
221933  jeffrey s. nelsen       checkerboard pr   
192145          wolff ryp  troll communications   
129214     denise hampton                  avon   

                                              book_oneStr  
64368   enter my jungle (harlequin presents) lindsay a...  
88742   doc and fluff: the dystopian tale of a girl an...  
221933  jungle jokes &amp

In [131]:
# Testing 3
user_input = 'Crime'
recommendations = get_recommendations(user_input)
print("Here are the Top 5 books recommendation for you:")
print(recommendations)

Here are the Top 5 books recommendation for you:
              ISBN                                              title  \
215320  0373287062           king's man (harlequin historical no 706)   
202578  0843936436                       morning rose, evening savage   
38723   0304352411       cross of iron (cassell military classics s.)   
11513   3792000261          der kleine prinz prince german hardy boys   
107108  041315310X  lesbian plays: any woman can, double vision, c...   

                           author          publisher  \
215320              caryn cameron          harlequin   
202578                 amii lorin      leisure books   
38723              willi heinrich   cassell military   
11513   antoine de saint-exupã©ry  karl rauch verlag   
107108                 jill davis            methuen   

                                              book_oneStr  
215320  king's man (harlequin historical no 706) caryn...  
202578  morning rose, evening savage amii lorin leisur.